In [1]:
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
from ThreeDCNN_LSTM import CNNLSTM, train
import torch.optim as optim
import torch.nn as nn
from NeuroimagingDataset import NeuroimagingDataset
import numpy as np

# Configuration options
# Define the number of folds and batch size
k_folds = 5
batch_size = 16

# Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the k-fold cross validation
kf = KFold(n_splits=k_folds, shuffle=True)

train_dataset = NeuroimagingDataset('/Users/weavejul/Documents/Class_and_Lab/My_Lab_Spring_24/Data/COBRE/ResultsS',
                                    '/Users/weavejul/Documents/Class_and_Lab/My_Lab_Spring_24/Data/COBRE/phenotypic_data.csv',
                                    feature_types='ALFF')

/Users/weavejul/anaconda3/envs/Torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Loop through each fold
for fold, (train_idx, test_idx) in enumerate(kf.split(train_dataset)):
    print(f"Fold {fold + 1}")
    print("-------")
    
    # get rid of bad indices
    train_idx = np.array([idx for idx in train_idx if idx not in [70, 83]])
    test_idx = np.array([idx for idx in test_idx if idx not in [70, 83]])

    # Define the data loaders for the current fold
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        sampler=torch.utils.data.SubsetRandomSampler(train_idx),
    )
    test_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        sampler=torch.utils.data.SubsetRandomSampler(test_idx),
    )

    # Initialize the model and optimizer
    model = CNNLSTM().to(device)
    #optimizer = optim.Adam(model.parameters(), lr=0.0005)
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    criterion = nn.CrossEntropyLoss()

    # Train the model on the current fold
    N_EPOCHS=10
    train(model, device, train_loader, optimizer, criterion, N_EPOCHS)

    # Evaluate the model on the test set
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.unsqueeze(1) # Unsqueeze for consistency!
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target) # TODO: crossentropy here?
            print(output)
            print("target")
            print(target)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100.0 * correct / len(test_loader.dataset)

    # Print the results for the current fold
    print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n")

Fold 1
-------

Epoch: 0
Batch labels: tensor([0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0])
Batch labels: tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0])
Batch labels: tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0])
Batch labels: tensor([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0])
Batch labels: tensor([1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0])
Batch labels: tensor([0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0])
Batch labels: tensor([0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0])
Batch labels: tensor([1, 0, 1, 1])
tensor([[0.1551, 0.0379],
        [0.1536, 0.0307],
        [0.1531, 0.0413],
        [0.1562, 0.0318],
        [0.1556, 0.0347],
        [0.1510, 0.0267],
        [0.1677, 0.0773],
        [0.1531, 0.0367],
        [0.1528, 0.0292],
        [0.1573, 0.0428],
        [0.1866, 0.1204],
        [0.1558, 0.0386],
        [0.1520, 0.0361],
        [0.1598, 0.0314],
        [0.1591, 0.0296],
        [0.1520, 0.0286]])
target
tensor([0, 1, 1, 1,